### Initialization

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container  width:90% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import sys
sys.path.insert(0, '../src')
import time
import argparse
import yaml
import types, copy, pprint
from time import sleep
from datetime import datetime
import pandas as pd
import numpy  as np
from utils import (initialize, init_dataloaders, init_environment, init_wandb, training_initializations, model_initializations, 
                   check_for_resume_training, disp_dataloader_info, disp_info_1, warmup_phase, weight_policy_training, 
                   display_gpu_info, init_dataloaders_by_fold_id, print_separator, print_heading, print_underline,  
                   timestring, print_loss, get_command_line_args, load_from_pickle) 

pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
pd.options.display.width = 132
# torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, profile=None, sci_mode=None)
os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Train_Resume.ipynb"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

## Create Environment

### Parse Input Args

In [3]:
# synthetic_5task_config = "../yamls/chembl_synt_train_5task.yaml"
# synthetic_config_file  = "../yamls/chembl_synt_train.yaml"
# config_file      = "../yamls/chembl_mini_train.yaml"
# config_file      = "../yamls/chembl_cb29_train_1task.yaml"
config_file      = "../yamls/chembl_cb29_train_10task.yaml"
batch_size=4098
exp_name  = "0906_1312"
# batch_size=2048
# RESUME_MODEL_CKPT = 'model_train_ep_25_seed_0088'

In [4]:
restart_input_args = f" --config  {config_file} " \
             f" --batch_size       {batch_size} "  \
             " --exp_desc            10-task with policy training " \
             " --hidden_size         4000 4000 4000 4000 4000 4000 "  \
             " --tail_hidden_size    4000 "  \
             " --warmup_epochs              20 "  \
             " --first_dropout            0.80 "  \
             " --middle_dropout           0.80 "  \
             " --last_dropout             0.80 "  \
             " --seed_idx                    0 "  \
             " --task_lr                 0.001 "  \
             " --backbone_lr             0.001 "  \
             " --decay_lr_rate             0.5 "  \
             " --decay_lr_freq              20 "  \
             " --decay_lr_cooldown           5 "  \
             " --policy_lr                0.01 "  \
             " --policy_decay_lr_rate      0.5 "  \
             " --policy_decay_lr_freq       20 "  \
             " --policy_decay_lr_cooldown    5 "  \
             " --lambda_tasks              1.0 "  \
             " --lambda_sparsity         0.001 "  \
             " --lambda_sharing           0.05 "  \
             " --pytorch_threads             7 "  \
             " --cuda_devices                2 "  \
             " --gpu_ids                     0 "  \
             " --resume "                         \
            f" --resume_path             ../../experiments/AdaSparseChem-cb29-10task/4000x6_{exp_name}_lr0.001_do0.8" \
             " --resume_ckpt             model_warmup_final_ep_78" \
             " --resume_metrics          metrics_warmup_final_ep_78" \
            f" --exp_name                {exp_name} " \
             " --folder_sfx              RETRAIN_1 "

#              " --exp_id                  154b30qo" \

In [5]:
ns = types.SimpleNamespace()
# input_args = input_args.split() if input_args is not None else input_args
input_args = restart_input_args.split() 
ns.args = get_command_line_args(input_args, display = True)
os.environ["CUDA_VISIBLE_DEVICES"]=ns.args.cuda_devices


 command line parms : 
------------------------
 config...................  ../yamls/chembl_cb29_train_10task.yaml
 project_name.............  None
 exp_id...................  1fgdsfry
 exp_name.................  0906_1312
 folder_sfx...............  RETRAIN_1
 exp_desc.................  10-task with policy training
 hidden_sizes.............  [4000, 4000, 4000, 4000, 4000, 4000]
 tail_hidden_size.........  [4000]
 warmup_epochs............  20
 training_epochs..........  None
 seed_idx.................  0
 batch_size...............  4098
 first_dropout............  0.8
 middle_dropout...........  0.8
 last_dropout.............  0.8
 backbone_lr..............  0.001
 task_lr..................  0.001
 policy_lr................  0.01
 decay_lr_rate............  0.5
 decay_lr_freq............  20
 decay_lr_cooldown........  5
 policy_decay_lr_rate.....  0.5
 policy_decay_lr_freq.....  20
 policy_decay_lr_cooldown.  5
 lambda_tasks.............  1.0
 lambda_sparsity..........  0.001
 lamb

In [6]:
# display_gpu_info()

### Read Configuration File

In [7]:

opt = initialize(ns, build_folders = True)


##################################################
################### READ YAML ####################
##################################################
 Pytorch thread count: 20
 Set Pytorch thread count to : 7
 Pytorch thread count set to : 7


wandb: Currently logged in as: kbardool. Use `wandb login --relogin` to force relogin


 WandB Initialization -----------------------------------------------------------
 PROJECT NAME: AdaSparseChem-cb29-10Task
 RUN ID      : 1fgdsfry 
 RUN NAME    : 0906_1312_RETRAIN_1
 --------------------------------------------------------------------------------


 log_dir              folder exists:  ../../experiments/AdaSparseChem-cb29-10task/4000x6_0906_1312_lr0.001_do0.8_RETRAIN_1
 result_dir           folder exists:  ../../experiments/AdaSparseChem-cb29-10task/4000x6_0906_1312_lr0.001_do0.8_RETRAIN_1
 checkpoint_dir       folder exists:  ../../experiments/AdaSparseChem-cb29-10task/4000x6_0906_1312_lr0.001_do0.8_RETRAIN_1

------------------------------------------------------------------------------------------------------------------------
 experiment name       : 0906_1312_RETRAIN_1 
 experiment id         : 1fgdsfry 
 folder_name           : 4000x6_0906_1312_lr0.001_do0.8_RETRAIN_1 
 experiment description: 10-task with policy training
 Random seeds          : [88, 45, 50, 10

In [8]:
# ns.wandb_run.finish()
# ns.wandb_run.finish()

### Setup Dataloader and Model  

In [9]:
# dldrs = init_dataloaders(opt, verbose = False)
dldrs = init_dataloaders_by_fold_id(opt, verbose = False)
disp_dataloader_info(dldrs)

 Warmup folds    : [2, 3, 4]
 Weights folds   : [2, 3]
 Policy folds    : [4]
 Validation folds: [1]
---------------------------------------------------------------------------
Load label/Y file for task 1 - task group chembl_29_Y_tg_0_cols_472.npy
--------------------------------------------------------------------------- 

 Number of non-zero features in ecfp[0]:79

 Task 1 label file: 
    Total > +1  Labels :          0 
    Total   +1  Labels :      81937 
    Total   -1  Labels :     188511 
    Total < -1  Labels :          0 
    Total != 0  Labels :     270448

 Task 1 files pre-filtering : 
--------------------------------
X file : # Samples :  423736     # Features per Sample: 32000   
Y file : # Samples :  423736     # Labels per Sample  : 472  Y rows with populated labels: 32866  non zero cols: 81937

 Task 1 files post-filtering : 
---------------------------------
X file : # Samples :  254529     # Features per Sample: 32000 
Y file : # Samples :  254529     # Labels per


 Task 8 files post-filtering : 
---------------------------------
X file : # Samples :  254529     # Features per Sample: 32000 
Y file : # Samples :  254529     # Labels per Sample  : 148  Y rows with populated labels: 13262  non zero cols: 28925

Using 80 of 148 classification tasks for calculating aggregated metrics (AUCROC, F1_max, etc).
------------------------------------------------------------------------------
Load label/Y file for task 9 - task group chembl_29_Y_tg_1028_cols_344.npy
------------------------------------------------------------------------------ 

 Number of non-zero features in ecfp[0]:79

 Task 9 label file: 
    Total > +1  Labels :          0 
    Total   +1  Labels :     110249 
    Total   -1  Labels :     213195 
    Total < -1  Labels :          0 
    Total != 0  Labels :     323444

 Task 9 files pre-filtering : 
--------------------------------
X file : # Samples :  423736     # Features per Sample: 32000   
Y file : # Samples :  423736     # Labels


 Task 6 files post-filtering : 
---------------------------------
X file : # Samples :  168649     # Features per Sample: 32000 
Y file : # Samples :  168649     # Labels per Sample  : 184  Y rows with populated labels: 7179  non zero cols: 19397

Using 92 of 184 classification tasks for calculating aggregated metrics (AUCROC, F1_max, etc).
-----------------------------------------------------------------------------
Load label/Y file for task 7 - task group chembl_29_Y_tg_836_cols_224.npy
----------------------------------------------------------------------------- 

 Number of non-zero features in ecfp[0]:79

 Task 7 label file: 
    Total > +1  Labels :          0 
    Total   +1  Labels :      38227 
    Total   -1  Labels :      91904 
    Total < -1  Labels :          0 
    Total != 0  Labels :     130131

 Task 7 files pre-filtering : 
--------------------------------
X file : # Samples :  423736     # Features per Sample: 32000   
Y file : # Samples :  423736     # Labels per


 Task 4 files post-filtering : 
---------------------------------
X file : # Samples :  85880     # Features per Sample: 32000 
Y file : # Samples :  85880     # Labels per Sample  : 192  Y rows with populated labels: 4510  non zero cols: 10141

Using 111 of 192 classification tasks for calculating aggregated metrics (AUCROC, F1_max, etc).
----------------------------------------------------------------------------
Load label/Y file for task 5 - task group chembl_29_Y_tg_11_cols_620.npy
---------------------------------------------------------------------------- 

 Number of non-zero features in ecfp[0]:79

 Task 5 label file: 
    Total > +1  Labels :          0 
    Total   +1  Labels :     142158 
    Total   -1  Labels :     193933 
    Total < -1  Labels :          0 
    Total != 0  Labels :     336091

 Task 5 files pre-filtering : 
--------------------------------
X file : # Samples :  423736     # Features per Sample: 32000   
Y file : # Samples :  423736     # Labels per Sam

X file : # Samples :  423736     # Features per Sample: 32000   
Y file : # Samples :  423736     # Labels per Sample  : 624  Y rows with populated labels: 38131  non zero cols: 90665

 Task 2 files post-filtering : 
---------------------------------
X file : # Samples :  86274     # Features per Sample: 32000 
Y file : # Samples :  86274     # Labels per Sample  : 624  Y rows with populated labels: 8661  non zero cols: 20196

Using 258 of 624 classification tasks for calculating aggregated metrics (AUCROC, F1_max, etc).
---------------------------------------------------------------------------
Load label/Y file for task 3 - task group chembl_29_Y_tg_6_cols_688.npy
--------------------------------------------------------------------------- 

 Number of non-zero features in ecfp[0]:79

 Task 3 label file: 
    Total > +1  Labels :          0 
    Total   +1  Labels :     320094 
    Total   -1  Labels :     382164 
    Total < -1  Labels :          0 
    Total != 0  Labels :     70225


 Task 10 files post-filtering : 
----------------------------------
X file : # Samples :  86274     # Features per Sample: 32000 
Y file : # Samples :  86274     # Labels per Sample  : 72  Y rows with populated labels: 1916  non zero cols: 4391

Using 52 of 72 classification tasks for calculating aggregated metrics (AUCROC, F1_max, etc).
 dataloader preparation - set number of batches per warmup training epoch to: 63
 dataloader preparation - set number of batches per weight training epoch to: 42
 dataloader preparation - set number of batches per policy training epoch to: 21
 dataloader preparation - set number of batches per validation to           : 22

 trainset.y_class                                   :  [(254529, 472), (254529, 624), (254529, 688), (254529, 192), (254529, 620), (254529, 184), (254529, 224), (254529, 148), (254529, 344), (254529, 72)] 
 trainset1.y_class                                  :  [(168649, 472), (168649, 624), (168649, 688), (168649, 192), (168649, 620

## Setup Model  

In [10]:
environ = init_environment(ns, opt, is_train = True, display_cfg = True)

##################################################
############# CREATE THE ENVIRONMENT #############
##################################################
 device is  cuda:0
--------------------------------------------------
 SparseChem_Backbone  Ver: 1.0 Init() Start 
-------------------------------------------------- 

 layer config        : [1, 1, 1, 1, 1, 1] 
 skip residual layers: False   skip hidden layers  : False
 SparseChem_BackBone() Input Layer  - Input: 32000  output: 4000  non-linearity:<class 'torch.nn.modules.activation.ReLU'>
 Hidden layer 0 - Input: 4000   output:4000
    _make_layer() using block: <class 'models.sparsechem_backbone.SparseChemBlock'>
           input_size: 4000 output_sz: 4000  non_linearity: ReLU() dropout: 0.8 bias: True
           SparseChemBlock.init(): input_size: 4000 output_sz: 4000   non_linearity: ReLU() dropout: 0.8 bias: True
 Hidden layer 1 - Input: 4000   output:4000
    _make_layer() using block: <class 'models.sparsechem_backbone.SparseChe

In [11]:
# print(environ.networks['mtl-net'].logits)
print(environ.networks['mtl-net'].policys)
print(environ.policy1)

[None, None, None, None, None, None, None, None, None, None]


AttributeError: 'SparseChemEnv' object has no attribute 'policy1'

In [12]:
model_initializations(ns, opt, environ, policy_learning = False)

 Model optimizers defined . . . policy_learning: False
 Model schedulers defined . . . policy_learning: False
 Metrics CSV file header written . . . 
 Model initializations complete . . . 


## Initiate / Resume Training Prep

In [13]:
print(opt['train']['which_iter'])

warmup


In [14]:
check_for_resume_training(ns, opt, environ)

opt['train']['which_iter'] :  warmup
##################################################
################ Resume training #################
##################################################
 Resume training from folder : ../../experiments/AdaSparseChem-cb29-10task/4000x6_0906_1312_lr0.001_do0.8
 Resume checkpoint filename  : model_warmup_final_ep_78
 Resume metrics filename     : metrics_warmup_final_ep_78
=> loading snapshot from ../../experiments/AdaSparseChem-cb29-10task/4000x6_0906_1312_lr0.001_do0.8/model_warmup_final_ep_78.tar
=> loading snapshot to   cuda:0
   Loading to GPU cuda:0
  networks -  network:  mtl-net
  load snapshot - network:  mtl-net
    network mtl-net - item task1_logits - shape: torch.Size([6, 2])
    network mtl-net - item task2_logits - shape: torch.Size([6, 2])
    network mtl-net - item task3_logits - shape: torch.Size([6, 2])
    network mtl-net - item task4_logits - shape: torch.Size([6, 2])
    network mtl-net - item task5_logits - shape: torch.Size([6, 

In [16]:
environ.load_policy(label = 'policy_warmup_final_ep_78', path = opt['resume_path'], verbose = True)

 load_policy(): load policies from ../../experiments/AdaSparseChem-cb29-10task/4000x6_0906_1312_lr0.001_do0.8/policy_warmup_final_ep_78.pickle
setting policy1 attribute .... to None
setting policy2 attribute .... to None
setting policy3 attribute .... to None
setting policy4 attribute .... to None
setting policy5 attribute .... to None
setting policy6 attribute .... to None
setting policy7 attribute .... to None
setting policy8 attribute .... to None
setting policy9 attribute .... to None
setting policy10 attribute .... to None


[None, None, None, None, None, None, None, None, None, None]

In [17]:
# print(environ.networks['mtl-net'].logits)
print(environ.networks['mtl-net'].policys)
print(environ.policy1)
print(environ.policy10)

[None, None, None, None, None, None, None, None, None, None]
None
None


In [22]:
# for k, v in environ.optimizers.items():
#     print(f' key: {k}  values: {v}')
#     print(v.state_dict())    

In [16]:
# for k, v in environ.schedulers.items():
#     print(f' key: {k}  values: {v}')
#     print(v.state_dict())

## Training Preparation

### Training Preparation

In [25]:
# print(f"\n folder: {opt['exp_folder']}",
#       f"\n layers: {opt['hidden_sizes']}",    
#       f"                               \n",
#       f"\n diff_sparsity_weights  : {opt['diff_sparsity_weights']}",
#       f"\n skip_layer             : {opt['skip_layer']}",
#       f"\n is_curriculum          : {opt['is_curriculum']}",
#       f"\n curriculum_speed       : {opt['curriculum_speed']}",
#       f"                              \n",    
#       f"\n decay_lr_rate          : {opt['train']['decay_lr_rate']}",      
#       f"\n decay_lr_freq          : {opt['train']['decay_lr_freq']}",     
#       f"\n policy_decay_lr_rate   : {opt['train']['policy_decay_lr_rate']}",      
#       f"\n policy_decay_lr_freq   : {opt['train']['policy_decay_lr_freq']}", 
#       f"                              \n",    
#       f"\n policy_lr              : {opt['train']['policy_lr']}", 
#       f"\n lambda_sparsity        : {opt['train']['lambda_sparsity']}",      
#       f"\n lambda_sharing         : {opt['train']['lambda_sharing']}", 
#       f"                              \n",    
#       f"\n lambda_tasks           : {opt['train']['lambda_tasks']}",  
#       f"\n init_temp              : {opt['train']['init_temp']}",
#       f"\n decay_temp             : {opt['train']['decay_temp']}",    
#       f"\n decay_temp_freq        : {opt['train']['decay_temp_freq']}",   
#       f"\n init_method            : {opt['train']['init_method']}", 
#       f"\n init_neg_logits        : {opt['train']['init_neg_logits']}",    
#       f"\n hard_sampling          : {opt['train']['hard_sampling']}",
#       f"\n Warm-up epochs         : {opt['train']['warm_up_epochs']}",
#       f"\n training epochs        : {opt['train']['training_epochs']}")

## Resume Warmup Training

### Training Preparation

In [18]:
print( f" Backbone Initial LR            :      {environ.opt['train']['backbone_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[0]['lr']} \n"
       f" Tasks    Initial LR            :      {environ.opt['train']['task_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[1]['lr']}    \n"
       f" Policy   Initial LR            :      {environ.opt['train']['policy_lr']:4f}      Current LR : {environ.optimizers['alphas'].param_groups[0]['lr']}  \n")

print( f" Backbone (Group 0) Initial LR  : {environ.opt['train']['backbone_lr']:4f} \n"
       f" Tasks    (Group 1) Initial LR  : {environ.opt['train']['task_lr']:4f}    \n Params : {environ.optimizers['weights']} \n\n"
       f" Policy   Initial LR            : {environ.opt['train']['policy_lr']:4f}  \n Params : {environ.optimizers['alphas']}  \n\n")

print( f" Backbone Initial LR            : {environ.opt['train']['backbone_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[0]['lr']} \n"
       f" Tasks    Initial LR            : {environ.opt['train']['task_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[1]['lr']}    \n"
       f" Policy   Initial LR            : {environ.opt['train']['policy_lr']:4f}      Current LR : {environ.optimizers['alphas'].param_groups[0]['lr']}  \n")



print_underline('Weights Scheduler Parameters', verbose = True) 
for k,i in environ.schedulers['weights'].state_dict().items():
    print(f"    {k:20s}     value: {i}")

print_underline('Policy Scheduler Parameters', verbose = True)
for k,i in environ.schedulers['alphas'].state_dict().items():
    print(f"    {k:20s}     value: {i}")    

 Backbone Initial LR            :      0.001000      Current LR : 0.00025 
 Tasks    Initial LR            :      0.001000      Current LR : 0.00025    
 Policy   Initial LR            :      0.010000      Current LR : 0.01  

 Backbone (Group 0) Initial LR  : 0.001000 
 Tasks    (Group 1) Initial LR  : 0.001000    
 Params : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.00025
    weight_decay: 0.0001

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.00025
    weight_decay: 0.0001
) 

 Policy   Initial LR            : 0.010000  
 Params : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0.0005
)  


 Backbone Initial LR            : 0.001000      Current LR : 0.00025 
 Tasks    Initial LR            : 0.001000      Current LR : 0.00025    
 Policy   Initial LR            : 0.010000      Current LR : 0.01  


Weights Scheduler Parameters
--

In [19]:
# print(environ.networks['mtl-net'].logits)
print(environ.networks['mtl-net'].policys)
print(environ.policy1)
print(environ.policy10)

[None, None, None, None, None, None, None, None, None, None]
None
None


In [21]:
training_initializations(ns, opt, environ, dldrs,  warmup = True)

 training preparation: - check for CUDA - cuda available as device id: [0]
 --> sparsechem_env.cuda()
 policy policy1 is None
 policy policy2 is None
 policy policy3 is None
 policy policy4 is None
 policy policy5 is None
 policy policy6 is None
 policy policy7 is None
 policy policy8 is None
 policy policy9 is None
 policy policy10 is None
dict_keys(['weights', 'alphas'])
 opt:  weights  
 item [step] not a Tensor ... 
 key: exp_avg               value: <class 'torch.Tensor'>   shape: torch.Size([472, 4000])  location: cpu
 item [exp_avg] is  Tensor - move  to cuda ... 
 key: exp_avg               value: <class 'torch.Tensor'>   shape: torch.Size([472, 4000])  location: cuda:0
 key: exp_avg_sq            value: <class 'torch.Tensor'>   shape: torch.Size([472, 4000])  location: cpu
 item [exp_avg_sq] is  Tensor - move  to cuda ... 
 key: exp_avg_sq            value: <class 'torch.Tensor'>   shape: torch.Size([472, 4000])  location: cuda:0
 item [step] not a Tensor ... 
 key: exp_avg   

 key: exp_avg_sq            value: <class 'torch.Tensor'>   shape: torch.Size([4000])  location: cuda:0
 item [step] not a Tensor ... 
 key: exp_avg               value: <class 'torch.Tensor'>   shape: torch.Size([4000, 4000])  location: cpu
 item [exp_avg] is  Tensor - move  to cuda ... 
 key: exp_avg               value: <class 'torch.Tensor'>   shape: torch.Size([4000, 4000])  location: cuda:0
 key: exp_avg_sq            value: <class 'torch.Tensor'>   shape: torch.Size([4000, 4000])  location: cpu
 item [exp_avg_sq] is  Tensor - move  to cuda ... 
 key: exp_avg_sq            value: <class 'torch.Tensor'>   shape: torch.Size([4000, 4000])  location: cuda:0
 item [step] not a Tensor ... 
 key: exp_avg               value: <class 'torch.Tensor'>   shape: torch.Size([4000])  location: cpu
 item [exp_avg] is  Tensor - move  to cuda ... 
 key: exp_avg               value: <class 'torch.Tensor'>   shape: torch.Size([4000])  location: cuda:0
 key: exp_avg_sq            value: <class 'torch

In [22]:
print(f" Sparsity regularization                    : {environ.opt['train']['lambda_sparsity']}\n"
      f" Sharing  regularization                    : {environ.opt['train']['lambda_sharing']} \n"
      f" Tasks    regularization                    : {environ.opt['train']['lambda_tasks']}   \n"
      f" Gumbel Temp                                : {environ.gumbel_temperature:.6f}         \n" #
      f" Gumbel Temp decay                          : {environ.opt['train']['decay_temp_freq']}") #

 Sparsity regularization                    : 0.001
 Sharing  regularization                    : 0.05 
 Tasks    regularization                    : 1.0   
 Gumbel Temp                                : 2.500000         
 Gumbel Temp decay                          : 3


In [24]:
print(f"current_epoch          : {ns.current_epoch}") 
print(f"current_iters          : {ns.current_iter}")  
print(f"train_total_epochs     : {ns.training_epochs}") 
print()
print(f"Batches in warmup epoch (trn_iters_warmup) : {ns.trn_iters_warmup}")
print(f"Batches in weight epoch (trn_iters_weight) : {ns.trn_iters_weights}")
print(f"Batches in policy epoch (trn_iters_policy) : {ns.trn_iters_policy}")
print(f"Batches in validation epoch (ns.eval_iters): {ns.eval_iters}")
print()

print_heading(f" Last Epoch: {ns.current_epoch}   # of warm-up epochs to do:  {ns.training_epochs} - Run epochs {ns.current_epoch+1} to {ns.current_epoch + ns.training_epochs}", verbose = True)


print_loss(ns.val_metrics, title = f"[e] Last ep:{ns.current_epoch}  it:{ns.current_iter} ")

print_heading(f" Last Epoch Completed : {ns.current_epoch}       # of epochs to run:  {ns.training_epochs} -->  epochs {ns.current_epoch+1} to {ns.training_epochs + ns.current_epoch}"
              f"\n Backbone Initial LR  : {environ.opt['train']['backbone_lr']}      Current LR : {environ.optimizers['weights'].param_groups[0]['lr']} "
              f"\n Heads    Initial LR  : {environ.opt['train']['task_lr']}      Current LR : {environ.optimizers['weights'].param_groups[1]['lr']}"              
              f"\n Policy   Initial LR  : {environ.opt['train']['policy_lr']}      Current LR : {environ.optimizers['alphas'].param_groups[0]['lr']}"
              f"\n Regularization tasks : {environ.opt['train']['lambda_tasks']}          Sparsity: {environ.opt['train']['lambda_sparsity']}           sharing: {environ.opt['train']['lambda_sharing']}"
              f"\n curriculum training  : {opt['is_curriculum']}      Cirriculum speed: {opt['curriculum_speed']}     num_training_layers : {ns.num_train_layers}", 
              verbose = True)

print()
environ.display_trained_logits(ns.current_epoch)
# environ.display_current_policy(ns.current_epoch)

current_epoch          : 78
current_iters          : 4914
train_total_epochs     : 250

Batches in warmup epoch (trn_iters_warmup) : 63
Batches in weight epoch (trn_iters_weight) : 42
Batches in policy epoch (trn_iters_policy) : 21
Batches in validation epoch (ns.eval_iters): 22

----------------------------------------------------------------------------
 Last Epoch: 78   # of warm-up epochs to do:  250 - Run epochs 79 to 328
---------------------------------------------------------------------------- 

[e] Last ep:78  it:4914  -  Losses:   	 Task: 2.2534   	 Sparsity: 1.41826e-05    	 Sharing: 5.65983e-06    	 Total: 2.2535 
------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 78       # of epochs to run:  250 -->  epochs 79 to 328
 Backbone Initial LR  : 0.001      Current LR : 0.00025 
 Heads    Initial LR  : 0.001      Current LR : 0.00025
 Policy   Initial LR  : 0.01      Current LR : 0.01

### Resume Warmup Training

In [29]:

warmup_phase(ns,opt, environ, dldrs, epochs = 50, verbose = False, disable_tqdm = False)


-----------------------------------------------------------------------------
 Last Epoch: 100   # of warm-up epochs to do:  50 - Run epochs 101 to 150
----------------------------------------------------------------------------- 

 Ep  | Trunk LR  Heads LR  Polcy LR  Gmbl Tmp |  trn tsk    trn spar    trn shar   trn ttl |    logloss   bceloss  avg prec    aucroc     aucpr    f1_max |  val tsk    val spar    val shar     total |  time |
 101 | 1.25e-04  1.25e-04  1.00e-02  2.50e+00 |   0.8247   1.530e-05   3.053e-05    0.8248 |  4.519e-06   0.50004   0.66789   0.76590   0.65444   0.69622 |   2.3441   3.073e-06   6.132e-06    2.3441 |  55.8 |
 102 | 1.25e-04  1.25e-04  1.00e-02  2.50e+00 |   0.7696   1.530e-05   3.053e-05    0.7697 |  4.547e-06   0.50393   0.66754   0.76562   0.65400   0.69648 |   2.3588   3.073e-06   6.132e-06    2.3588 |  58.6 |
 103 | 1.25e-04  1.25e-04  1.00e-02  2.50e+00 |   0.7835   1.530e-05   3.053e-05    0.7835 |  4.583e-06   0.50256   0.66757   0.76597   0.653

In [35]:
import torch
for o in environ.optimizers.keys():
    for k, state in environ.optimizers[o].state.items():
        print("------------------------------------------------------------------------------------------------------------------")
        print(f' optimizer: {o}    key: {k.shape}   type: {type(k)}   device: {k.device}  value: {type(state)}')
#         print(k)
        print("------------------------------------------------------------------------------------------------------------------")
    #     print(state)

        for kk, vv in state.items():
            if torch.is_tensor(vv) :
                print(f' key: {kk:20s}  value: {type(vv)}   shape: {vv.shape}  location: {vv.device}')
            else:
                print(f' key: {kk:20s}  value: {vv}         shape: --       location: --')
#             print(vv)
     

------------------------------------------------------------------------------------------------------------------
 optimizer: weights    key: torch.Size([472, 4000])   type: <class 'torch.nn.parameter.Parameter'>   device: cuda:0  value: <class 'dict'>
------------------------------------------------------------------------------------------------------------------
 key: step                  value: 4914         shape: --       location: --
 key: exp_avg               value: <class 'torch.Tensor'>   shape: torch.Size([472, 4000])  location: cuda:0
 key: exp_avg_sq            value: <class 'torch.Tensor'>   shape: torch.Size([472, 4000])  location: cuda:0
------------------------------------------------------------------------------------------------------------------
 optimizer: weights    key: torch.Size([472])   type: <class 'torch.nn.parameter.Parameter'>   device: cuda:0  value: <class 'dict'>
----------------------------------------------------------------------------------------

## Resume Weight/Policy Training

### Training Preparation

In [14]:
disp_dataloader_info(dldrs)


 trainset.y_class                                   :  [(254529, 472), (254529, 624), (254529, 688), (254529, 192), (254529, 620), (254529, 184), (254529, 224), (254529, 148), (254529, 344), (254529, 72)] 
 trainset1.y_class                                  :  [(168649, 472), (168649, 624), (168649, 688), (168649, 192), (168649, 620), (168649, 184), (168649, 224), (168649, 148), (168649, 344), (168649, 72)] 
 trainset2.y_class                                  :  [(85880, 472), (85880, 624), (85880, 688), (85880, 192), (85880, 620), (85880, 184), (85880, 224), (85880, 148), (85880, 344), (85880, 72)] 
 valset.y_class                                     :  [(86274, 472), (86274, 624), (86274, 688), (86274, 192), (86274, 620), (86274, 184), (86274, 224), (86274, 148), (86274, 344), (86274, 72)]  

                                Total                :  595332 


Training dataset :
--------------------
  Size of training set 0 (warm up)                   :  254529 
  Number of batches in 

In [17]:
ns.flag = 'update_weights'


 Model optimizers defined . . . policy_learning: True
 Model schedulers defined . . . policy_learning: True
 Metrics CSV file header written . . . 
 Model initializations complete . . . 


In [38]:
print( f" Backbone Initial LR            :      {environ.opt['train']['backbone_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[0]['lr']} \n"
       f" Tasks    Initial LR            :      {environ.opt['train']['task_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[1]['lr']}    \n"
       f" Policy   Initial LR            :      {environ.opt['train']['policy_lr']:4f}      Current LR : {environ.optimizers['alphas'].param_groups[0]['lr']}  \n")

print( f" Backbone (Group 0) Initial LR  : {environ.opt['train']['backbone_lr']:4f} \n"
       f" Tasks    (Group 1) Initial LR  : {environ.opt['train']['task_lr']:4f}    \n Params : {environ.optimizers['weights']} \n\n"
       f" Policy   Initial LR            : {environ.opt['train']['policy_lr']:4f}  \n Params : {environ.optimizers['alphas']}  \n\n")

print( f" Backbone Initial LR            : {environ.opt['train']['backbone_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[0]['lr']} \n"
       f" Tasks    Initial LR            : {environ.opt['train']['task_lr']:4f}      Current LR : {environ.optimizers['weights'].param_groups[1]['lr']}    \n"
       f" Policy   Initial LR            : {environ.opt['train']['policy_lr']:4f}      Current LR : {environ.optimizers['alphas'].param_groups[0]['lr']}  \n")

print_underline('Weights Scheduler Parameters', verbose = True) 
for k,i in environ.schedulers['weights'].state_dict().items():
    print(f"    {k:20s}     value: {i}")

print_underline('Policy Scheduler Parameters', verbose = True)
for k,i in environ.schedulers['alphas'].state_dict().items():
    print(f"    {k:20s}     value: {i}")    

 Backbone Initial LR            :      0.001000      Current LR : 0.001 
 Tasks    Initial LR            :      0.001000      Current LR : 0.001    
 Policy   Initial LR            :      0.010000      Current LR : 0.01  

 Backbone (Group 0) Initial LR  : 0.001000 
 Tasks    (Group 1) Initial LR  : 0.001000    
 Params : SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001

Parameter Group 1
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
) 

 Policy   Initial LR            : 0.010000  
 Params : Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0.0005
)  


 Backbone Initial LR            : 0.001000      Current LR : 0.001 
 Tasks    Initial LR            : 0.001000      Current LR : 0.001    
 Policy   Initial LR            : 0.010000      Current LR : 0.01  



In [49]:
# training_initializations(ns, opt, environ, dldrs, warmup_iterations = 200, weight_iterations = 200, policy_iterations = 200, eval_iterations = 50, warmup = False)
training_initializations(ns, opt, environ, dldrs,  warmup = False)


 training preparation: - check for CUDA - cuda available as device id: [0]
sparsechem_env.cuda()
 training preparation: - set print_freq to                                 : 63 
 training preparation: - set number of batches per warmup training epoch to: 63
 training preparation: - set number of batches per weight training epoch to: 42
 training preparation: - set number of batches per policy training epoch to: 21
 training preparation: - set number of batches per validation to           : 22
 training preparation complete . . .


In [27]:
print_heading( f"** {timestring()} - Training iteration {ns.current_iter}   flag: {ns.flag} \n"
               f"** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)\n"
               f"** Switch from Warm Up training to Alternate training Weights & Policy \n"
               f"** Take checkpoint and block gradient flow through Policy net", verbose=True)

print(environ.disp_for_excel())

environ.display_trained_policy(ns.current_epoch,out=sys.stdout)
environ.display_trained_logits(ns.current_epoch,out=sys.stdout)

------------------------------------------------------------------------------------------------------------------------
** 2022-09-05 23:15:36:451140 - Training iteration 497250   flag: update_weights 
** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)
** Switch from Warm Up training to Alternate training Weights & Policy 
** Take checkpoint and block gradient flow through Policy net
------------------------------------------------------------------------------------------------------------------------ 


    folder: 4000x6_0829_2050_lr0.001_do0.8_RETRAIN_1
    layers: 6 [4000, 4000, 4000, 4000, 4000, 4000] 
    
    first dropout          : 0.8
    middle dropout         : 0.8
    last dropout           : 0.8
    diff_sparsity_weights  : False
    skip_layer             : 0
    is_curriculum          : False
    curriculum_speed       : 3
    
    task_lr                : 0.001
    backbone_lr            : 0.001
    decay_lr_rate      

### Resume Training

In [29]:
print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.6f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

print(f"Batches in warmup epoch (trn_iters_warmup) : {ns.trn_iters_warmup}")
print(f"Batches in weight epoch (trn_iters_weight) : {ns.trn_iters_weights}")
print(f"Batches in policy epoch (trn_iters_policy) : {ns.trn_iters_policy}")
print(f"Batches in validation epoch (ns.eval_iters): {ns.eval_iters}")

 Sparsity regularization     : 0.001
 Sharing  regularization     : 0.05 
 Tasks    regularization     : 1.0   
 Gumbel Temp                 : 0.000188         
 Gumbel Temp decay           : 3
Batches in warmup epoch (trn_iters_warmup) : 200
Batches in weight epoch (trn_iters_weight) : 200
Batches in policy epoch (trn_iters_policy) : 200
Batches in validation epoch (ns.eval_iters): 50


In [34]:
environ.opt['train']['lambda_sparsity'] = 0.05
environ.opt['train']['lambda_sharing']  = 0.05
# environ.opt['train']['decay_temp_freq'] = 3
# ns.trn_iters_policy = 100
ns.training_epochs = 10

In [31]:
print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

 Sparsity regularization     : 0.05
 Sharing  regularization     : 0.05 
 Tasks    regularization     : 1.0   
 Gumbel Temp                 : 0.0002         
 Gumbel Temp decay           : 3


In [33]:
print(f"current_epoch          : {ns.current_epoch}") 
print(f"current_iters          : {ns.current_iter}")  
print(f"train_total_epochs     : {ns.training_epochs}") 
print(f"Batches in warmup epoch (trn_iters_warmup) : {ns.trn_iters_warmup}")
print(f"Batches in weight epoch (trn_iters_weight) : {ns.trn_iters_weights}")
print(f"Batches in policy epoch (trn_iters_policy) : {ns.trn_iters_policy}")
print(f"Batches in validation epoch (ns.eval_iters): {ns.eval_iters}")
print()

print_heading(f" Last Epoch: {ns.current_epoch}   # of warm-up epochs to do:  {ns.training_epochs} - Run epochs {ns.current_epoch+1} to {ns.current_epoch + ns.training_epochs}", verbose = True)


print_loss(ns.val_metrics, title = f"[e] Last ep:{ns.current_epoch}  it:{ns.current_iter} ")

print_heading(f" Last Epoch Completed : {ns.current_epoch}       # of epochs to run:  {ns.training_epochs} -->  epochs {ns.current_epoch+1} to {ns.training_epochs + ns.current_epoch}"
              f"\n Backbone Initial LR  : {environ.opt['train']['backbone_lr']}      Current LR : {environ.optimizers['weights'].param_groups[0]['lr']} "
              f"\n Heads    Initial LR  : {environ.opt['train']['task_lr']}      Current LR : {environ.optimizers['weights'].param_groups[1]['lr']}"              
              f"\n Policy   Initial LR  : {environ.opt['train']['policy_lr']}      Current LR : {environ.optimizers['alphas'].param_groups[0]['lr']}"
              f"\n Regularization tasks : {environ.opt['train']['lambda_tasks']}          Sparsity: {environ.opt['train']['lambda_sparsity']}           sharing: {environ.opt['train']['lambda_sharing']}"
              f"\n curriculum training  : {opt['is_curriculum']}      Cirriculum speed: {opt['curriculum_speed']}     num_training_layers : {ns.num_train_layers}", 
              verbose = True)

print()
environ.display_trained_logits(ns.current_epoch)
# environ.display_current_policy(ns.current_epoch)

current_epoch          : 250
current_iters          : 497250
train_total_epochs     : 250
Batches in warmup epoch (trn_iters_warmup) : 200
Batches in weight epoch (trn_iters_weight) : 200
Batches in policy epoch (trn_iters_policy) : 200
Batches in validation epoch (ns.eval_iters): 50

------------------------------------------------------------------------------
 Last Epoch: 250   # of warm-up epochs to do:  250 - Run epochs 251 to 500
------------------------------------------------------------------------------ 

[e] Last ep:250  it:497250  -  Losses:   	 Task: 2.1423   	 Sparsity: 9.96697e-04    	 Sharing: 1.38397e-03    	 Total: 2.1447 
------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 250       # of epochs to run:  250 -->  epochs 251 to 500
 Backbone Initial LR  : 0.001      Current LR : 0.001 
 Heads    Initial LR  : 0.001      Current LR : 0.001
 Policy   Initial LR  : 0.01      Curre

In [88]:
weight_policy_training(ns, opt, environ, dldrs, display_policy = True, disable_tqdm = False, epochs = 6)

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 42   # of epochs to run:  2 -->  epochs 43 to 44    
 policy_learning rate : 0.001      
 lambda_sparsity      : 0.0
 lambda_sharing       : 0.001 
 curriculum training  : False     cirriculum speed: 3     num_training_layers : 6
------------------------------------------------------------------------------------------------------------------------ 

 Ep  | Trunk LR  Heads LR  Polcy LR  Gmbl Tmp |  trn tsk    trn spar    trn shar   trn ttl |   logloss   bceloss  avg prec    aucroc     aucpr    f1_max |  val tsk    val spar    val shar     total | time |     
  43 | 3.00e-04  3.00e-04  1.00e-03  2.25e-01 |   1.8536   0.000e+00   2.754e-04    1.8539 |   0.00006   0.51391   0.75470   0.75609   0.71561   0.79427 |   2.1611   0.000e+00   2.754e-04    2.1614 |195.1 |
  43 | 3.00e-04  3.00e-04  1.00e-03  2.25e-01 |   1.6060   0.000e+00   2.751e-04   

 save train checkpoint  to :  model_train_last_ep_44
 save train metrics to     :  metrics_train_last_ep_44.pickle
[Final] ep:44  it:30090 -  Total Loss: 2.2659     
Task: 2.2653   Sparsity: 0.00000e+00    Sharing: 6.29763e-04 

 ep:   44   logits       s         logits       s         logits       s         logits       s         logits       s         logits       s         logits       s         logits       s         logits       s         logits       s         
 ----- ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    
  0   -0.1265    0.0540  0   -0.0957    0.0353  0   -0.4478    0.4996  0   -0.2430    0.2604  0   -0.3036    0.3471  0   -0.0283   -0.0255  0   -0.1234    0.1157  0   -0.2221    0.2423  0   -0.2658    0.3133  0   -0.0782    0.0251  0
  1   -0.1002    0.0679  0   -0.0473   -0.0317  0   -0.4

In [72]:
print(environ.policy1)
print(environ.networks['mtl-net'].policys[0])

tensor([[0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0]], device='cuda:0')
tensor([[0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0]], device='cuda:0')


In [51]:
print(environ.logit1)
print(environ.networks['mtl-net'].task1_logits)

array([[-0.10928034,  0.03533224],
       [-0.11588797,  0.08241694],
       [-0.08529188,  0.02049774],
       [-0.08840536,  0.03978787],
       [-0.1570164 ,  0.12076251],
       [-0.1639516 ,  0.18603538]], dtype=float32)

In [85]:
environ.display_trained_logits()
environ.display_trained_policy()

environ.display_current_policy()


 ep:    0   logits       s         logits       s         logits       s         logits       s         logits       s         logits       s         logits       s         logits       s         logits       s         logits       s         
 ----- ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    ----------------- -    
  0   -0.1093    0.0353  0   -0.0769    0.0174  0   -0.4064    0.4563  0   -0.2180    0.2339  0   -0.3018    0.3437  0   -0.0600    0.0115  0   -0.1294    0.1227  0   -0.1851    0.2055  0   -0.2733    0.3198  0   -0.0879    0.0374  0
  1   -0.1159    0.0824  0   -0.0333   -0.0434  1   -0.3981    0.4598  0   -0.0347    0.0106  0   -0.2812    0.3267  0   -0.1059    0.0570  0   -0.1659    0.1415  0   -0.1193    0.1536  0   -0.1683    0.2030  0   -0.0852    0.0408  0
  2   -0.0853    0.0205  0   -0.0473   -0.0371  0 

In [75]:
ns.wandb_run.finish()

auc_pr,▆█▅▆█▁▆▁▅▅
avg_prec_score,▇█▅▇█▂▇▁▅▅
bceloss,▆▄▇▇▁█▆▃▄▂
epoch,▁▂▃▃▄▅▆▆▇█
f1_max,█▇▅▇▇▃█▁▆▅
gumbel_temp,███▆▆▆▃▃▃▁
kappa,▅▂▄▄█▄▁▂▅▅
kappa_max,▆▇▅▅█▃▇▁▇▅
lambda_sharing,▁▁▁▁▁▁▁▁▁▁
lambda_sparsity,▁▁▁▁▁▁▁▁▁▁
lambda_tasks,▁▁▁▁▁▁▁▁▁▁


## Post Training Stuff

In [65]:
environ.schedulers

{'alphas': <torch.optim.lr_scheduler.ReduceLROnPlateau at 0x7f7d1c787e50>,
 'weights': <torch.optim.lr_scheduler.ReduceLROnPlateau at 0x7f7d1c787e20>}

In [ ]:
from torch import nn
current_state = {}
for k, v in environ.networks.items():
    print(f' k: {k}')
    if isinstance(v, nn.DataParallel):
        current_state[k] = v.module.state_dict()
    else:
        current_state[k] = v.state_dict()
        
print(current_state.keys())

for k, v in environ.optimizers.items():
    print(f' k: {k}')
    current_state[k] = v.state_dict()
    

In [69]:
for k, v in environ.schedulers.items():
    print(f' k: {k}    v: {v}')
    current_state[k] = v.state_dict()
    print(current_state[k])
# for k in current_state:
#     print(f'key: {k}', '\n', current_state[k].keys(), '\n')    
    

 k: alphas    v: <torch.optim.lr_scheduler.ReduceLROnPlateau object at 0x7f7d1c787e50>
{'factor': 0.5, 'min_lrs': [0], 'patience': 20, 'verbose': True, 'cooldown': 5, 'cooldown_counter': 0, 'mode': 'min', 'threshold': 0.0001, 'threshold_mode': 'rel', 'best': inf, 'num_bad_epochs': 0, 'mode_worse': inf, 'eps': 1e-08, 'last_epoch': 0}
 k: weights    v: <torch.optim.lr_scheduler.ReduceLROnPlateau object at 0x7f7d1c787e20>
{'factor': 0.5, 'min_lrs': [0, 0], 'patience': 20, 'verbose': True, 'cooldown': 5, 'cooldown_counter': 0, 'mode': 'min', 'threshold': 0.0001, 'threshold_mode': 'rel', 'best': inf, 'num_bad_epochs': 0, 'mode_worse': inf, 'eps': 1e-08, 'last_epoch': 0}


In [36]:
print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}")
# environ.display_trained_policy(current_epoch)
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

[Final] ep:287  it:58757 -  Total Loss: 15.5793     
Task: 11.2231   Sparsity: 4.35058e+00    Sharing: 5.58868e-03 


In [246]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")


print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

print( f" current_iters               : {current_iter}")  
print( f" current_epochs              : {current_epoch}") 
print( f" train_total_epochs          : {train_total_epochs}") 
print( f" stop_epoch_training         : {stop_epoch_training}")

 Backbone Learning Rate      : 0.001
 Tasks    Learning Rate      : 0.001
 Policy   Learning Rate      : 0.01

 Sparsity regularization     : 0.0
 Sharing  regularization     : 0.0001 
 Tasks    regularization     : 1.0   
 Gumbel Temp                 : 0.7234         
 Gumbel Temp decay           : 2


In [247]:
environ.opt['train']['lambda_sparsity'] = 0.0000
environ.opt['train']['lambda_sharing']  = 0.001
environ.opt['train']['lambda_tasks']    = 1.0
environ.opt['train']['decay_temp_freq'] = 2

In [250]:
print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}\n") #


print( f" current_iters               : {current_iter}")  
print( f" current_epochs              : {current_epoch}") 
print( f" train_total_epochs          : {train_total_epochs}") 
print( f" stop_epoch_training         : {stop_epoch_training}")

 Sparsity regularization     : 0.0
 Sharing  regularization     : 0.001 
 Tasks    regularization     : 1.0   
 Gumbel Temp                 : 0.7234         
 Gumbel Temp decay           : 2

 current_iters               : 38669
 current_epochs              : 192
 train_total_epochs          : 5
 stop_epoch_training         : 192


In [17]:
# train_total_epochs = 50
stop_epoch_training = current_epoch + train_total_epochs     
print(f"current_iters         : {current_iter}")  
print(f"current_epochs        : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 
print(f"stop_epoch_training   : {stop_epoch_training}")

current_iters         : 10908
current_epochs        : 51
train_total_epochs    : 50
stop_epoch_training   : 101


In [ ]:
with np.printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', precision=7, formatter={'float': lambda x: f"{x:12.5e}"}):
    environ.print_logit_grads('gradients')


In [136]:
environ.display_trained_logits(current_epoch)
environ.display_trained_policy(current_epoch)


 epch: 312   logits        sel          logits       sel         logits        sel 
 -----  -----------------  ---    ----------------   ---    ----------------   --- 
   1   -0.4585     0.6026   0    -0.4601     0.8096   0    -0.4597     0.5298   0
   2   -0.1734    -0.9305   1    -0.1718    -1.0447   1    -0.1615    -0.9853   1
   3   -0.1602    -1.1804   1    -0.1241    -1.2051   1    -0.1391    -1.3729   1
   4   -0.3442     0.1300   0    -0.3490     0.0837   0    -0.3420     0.6391   0
   5   -0.2803     0.3260   0    -0.2921    -0.8131   1    -0.2676    -0.2554   0
   6   -0.4716     1.7158   0    -0.4718     1.5023   0    -0.4718     1.1004   0



 epch: 312   softmax       sel        softmax        sel        softmax        sel 
 -----  -----------------  ---    -----------------  ---    -----------------  --- 
   1    0.2571     0.7429   0     0.2193     0.7807   0     0.2710     0.7290   0
   2    0.6807     0.3193   1     0.7054     0.2946   1     0.6950     0.3050   1
   3

In [123]:
environ.display_test_sample_policy(current_epoch, hard_sampling = True)
environ.display_train_sample_policy(current_epoch, hard_sampling = True)

 Sample Policy (Testing mode - hard_sampling: True) 
 312 epochs  logits         sel        logits         sel         logits         sel 
 -----   ----------------  -----    ----------------  -----    ---------------   ----- 
   1    -0.4585    0.6026  [0 1]   -0.4601    0.8096  [0 1]   -0.4597    0.5298  [0 1]
   2    -0.1734   -0.9305  [1 0]   -0.1718   -1.0447  [1 0]   -0.1615   -0.9853  [1 0]
   3    -0.1602   -1.1804  [1 0]   -0.1241   -1.2051  [1 0]   -0.1391   -1.3729  [1 0]
   4    -0.3442    0.1300  [0 1]   -0.3490    0.0837  [0 1]   -0.3420    0.6391  [0 1]
   5    -0.2803    0.3260  [0 1]   -0.2921   -0.8131  [1 0]   -0.2676   -0.2554  [0 1]
   6    -0.4716    1.7158  [0 1]   -0.4718    1.5023  [0 1]   -0.4718    1.1004  [0 1]


 Sample Policy (Training mode - hard_sampling: True) 
 312 epochs    logits          gumbel                logits           gumbel               logits             gumbel 
 -----   ----------------------------------     -----------------------------

In [117]:
environ.display_test_sample_policy(current_epoch, hard_sampling = False)
environ.display_train_sample_policy(current_epoch, hard_sampling = False)

 Sample Policy (Testing mode - hard_sampling: False) 
 312 epochs  logits         sel        logits         sel         logits         sel 
 -----   ----------------  -----    ----------------  -----    ---------------   ----- 
   1    -0.4585    0.6026  [0 1]   -0.4601    0.8096  [0 1]   -0.4597    0.5298  [1 0]
   2    -0.1734   -0.9305  [1 0]   -0.1718   -1.0447  [0 1]   -0.1615   -0.9853  [1 0]
   3    -0.1602   -1.1804  [1 0]   -0.1241   -1.2051  [1 0]   -0.1391   -1.3729  [1 0]
   4    -0.3442    0.1300  [1 0]   -0.3490    0.0837  [0 1]   -0.3420    0.6391  [1 0]
   5    -0.2803    0.3260  [0 1]   -0.2921   -0.8131  [1 0]   -0.2676   -0.2554  [0 1]
   6    -0.4716    1.7158  [1 0]   -0.4718    1.5023  [0 1]   -0.4718    1.1004  [0 1]


 Sample Policy (Training mode - hard_sampling: False) 
 312 epochs    logits          gumbel                logits           gumbel               logits             gumbel 
 -----   ----------------------------------     ---------------------------

In [137]:
environ.opt['train']['hard_sampling']

True

In [9]:
# 
p = environ.get_current_state(0)

In [11]:
pp.pprint(p)

{   'alphas': {   'param_groups': [   {   'amsgrad': False,
                                          'betas': (0.9, 0.999),
                                          'eps': 1e-08,
                                          'initial_lr': 0.01,
                                          'lr': 0.01,
                                          'params': [0, 1, 2],
                                          'weight_decay': 0.0005}],
                  'state': {}},
    'iter': 0,
    'mtl-net': OrderedDict([   (   'task1_logits',
                                   tensor([[ 0.001544,  0.002121],
        [ 0.000495,  0.001111],
        [ 0.000426, -0.000401],
        [ 0.000426, -0.000470],
        [ 0.000497,  0.000899],
        [ 0.002776, -0.000226]])),
                               (   'task2_logits',
                                   tensor([[ 0.000482, -0.001070],
        [-0.000505, -0.001076],
        [ 0.000701, -0.000132],
        [-0.000171, -0.001668],
        [ 0.000262, -0.000324]

### Post Warm-up Training stuff

In [15]:
pp.pprint(environ.val_metrics)

{   'aggregated': {   'auc_pr': 0.5683542231355692,
                      'avg_prec_score': 0.5686955149511226,
                      'bceloss': 0.6940069357554117,
                      'f1_max': 0.6753742658177068,
                      'kappa': 0.08318090860174578,
                      'kappa_max': 0.11385237142207795,
                      'logloss': tensor(0.0002, device='cuda:0', dtype=torch.float64),
                      'p_f1_max': 0.29707588851451877,
                      'p_kappa_max': 0.48534467220306404,
                      'roc_auc_score': 0.5679242272843051,
                      'sc_loss': tensor(0.2891, device='cuda:0', dtype=torch.float64)},
    'epoch': 1,
    'loss': {   'task1': 3.5098743182605396,
                'task2': 3.4508372449718867,
                'task3': 3.4480013587242775,
                'total': 10.408712921956704},
    'loss_mean': {   'task1': 0.701974863652108,
                     'task2': 0.6901674489943774,
                     'task3': 0.

In [16]:
environ.networks['mtl-net'].arch_parameters()

[Parameter containing:
 tensor([[0.5000, 0.5000],
         [0.5000, 0.5000],
         [0.5000, 0.5000]], device='cuda:0'),
 Parameter containing:
 tensor([[0.5000, 0.5000],
         [0.5000, 0.5000],
         [0.5000, 0.5000]], device='cuda:0'),
 Parameter containing:
 tensor([[0.5000, 0.5000],
         [0.5000, 0.5000],
         [0.5000, 0.5000]], device='cuda:0')]

In [31]:
p = environ.get_sample_policy(hard_sampling = False)
print(p)
p = environ.get_policy_prob()
print(p)
p = environ.get_all_task_logits()
print(p)

# p = environ.get_current_policy()
# print(p)

([tensor([[1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1]], device='cuda:0'), tensor([[0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1]], device='cuda:0'), tensor([[1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1]], device='cuda:0')], [array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=float32), array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=float32), array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=float32)])
[array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=float32), array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=floa

In [42]:
a = softmax([0.0, 1])
print(a)
sampled = np.random.choice((1, 0), p=a)
print(sampled)

[0.26894142 0.73105858]
1


In [20]:
print(environ.optimizers['weights'])
print(environ.schedulers['weights'].get_last_lr())

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.5, 0.999)
    eps: 1e-08
    initial_lr: 0.01
    lr: 0.01
    weight_decay: 0.0001

Parameter Group 1
    amsgrad: False
    betas: (0.5, 0.999)
    eps: 1e-08
    initial_lr: 0.01
    lr: 0.01
    weight_decay: 0.0001
)
[0.01, 0.01]


In [17]:
print('losses.keys      : ', environ.losses.keys())
print('losses[task]keys : ', environ.losses['task1'].keys())
pp.pprint(environ.losses)

losses.keys      :  dict_keys(['parms', 'losses', 'losses_mean', 'sparsity', 'sharing', 'total', 'total_mean', 'task1', 'task2', 'task3'])
losses[task]keys :  dict_keys(['cls_loss', 'cls_loss_mean'])
{   'losses': {   'task1': tensor(3.5704, device='cuda:0', dtype=torch.float64),
                  'task2': tensor(3.3313, device='cuda:0', dtype=torch.float64),
                  'task3': tensor(3.3721, device='cuda:0', dtype=torch.float64),
                  'total': tensor(10.2737, device='cuda:0', dtype=torch.float64)},
    'losses_mean': {   'task1': tensor(0.7141, device='cuda:0', dtype=torch.float64),
                       'task2': tensor(0.6663, device='cuda:0', dtype=torch.float64),
                       'task3': tensor(0.6744, device='cuda:0', dtype=torch.float64),
                       'total': tensor(2.0547, device='cuda:0', dtype=torch.float64)},
    'parms': {'gumbel_temp': 5, 'lr_0': 0.01, 'lr_1': 0.01, 'train_layers': 0},
    'sharing': {'total': tensor(0., device='cuda:

In [20]:
print( environ.val_metrics.keys())
# pp.pprint(val_metrics)
print(type(environ.val_metrics['aggregated']))
print()
print(type(environ.val_metrics['task1']['classification_agg']))
print()
pp.pprint(environ.val_metrics)

dict_keys(['loss', 'loss_mean', 'task1', 'task2', 'task3', 'aggregated', 'train_time', 'epoch'])
<class 'dict'>

<class 'dict'>

{   'aggregated': {   'auc_pr': 0.8163426647572675,
                      'avg_prec_score': 0.8164138615123665,
                      'bceloss': 0.7729340473810831,
                      'f1_max': 0.7574111413660235,
                      'kappa': 0.4593877004350045,
                      'kappa_max': 0.4722816344851592,
                      'logloss': tensor(0.0002, device='cuda:0', dtype=torch.float64),
                      'p_f1_max': 0.1918067594369252,
                      'p_kappa_max': 0.545021508137385,
                      'roc_auc_score': 0.8101998985724014,
                      'sc_loss': tensor(0.3217, device='cuda:0', dtype=torch.float64)},
    'epoch': 4000,
    'loss': {   'task1': 3.990690022259455,
                'task2': 3.7726694706664947,
                'task3': 3.817710672775008,
                'total': 11.581070165700957},
    'l

In [21]:
print(f" val_metric keys               : {val_metrics.keys()}")
print(f" loss keys                     : {val_metrics['loss'].keys()}")
print(f" task1 keys                    : {val_metrics['task1'].keys()}")
print(f" task1 classification keys     : {val_metrics['task1']['classification'].keys()}")
print(f" task1 classification_agg keys : {val_metrics['task1']['classification_agg'].keys()}")
print()
print(f" task1                       : {val_metrics['task1']['classification_agg']['loss']:5f}")
print(f" task2                       : {val_metrics['task2']['classification_agg']['loss']:5f}")
print(f" task3                       : {val_metrics['task3']['classification_agg']['loss']:5f}")
print(f" loss                        : {val_metrics['loss']['total']:5f}")
print(f" train_time                  : {val_metrics['train_time']:2f}")
print(f" epoch                       : {val_metrics['epoch']}")


AttributeError: 'NoneType' object has no attribute 'keys'

### Post Weight + Policy Training Stuff 

In [178]:
environ.networks['mtl-net'].backbone.layer_config

[1, 1, 1, 1, 1, 1]

In [63]:
num_blocks = 6
num_policy_layers = 6
gt =  torch.ones((num_blocks)).long()
gt0 =  torch.zeros((num_blocks)).long()
print(gt)
print(gt0)

loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
print(loss_weights)

tensor([1, 1, 1, 1, 1, 1])
tensor([0, 0, 0, 0, 0, 0])
tensor([0.1667, 0.3333, 0.5000, 0.6667, 0.8333, 1.0000])


In [77]:
if environ.opt['diff_sparsity_weights'] and not environ.opt['is_sharing']:
    print(' cond 1')
    ## Assign higher weights to higher layers 
    loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
    print(f"{task_key} sparsity error:  {2 * (loss_weights[-num_blocks:] * environ.cross_entropy2(logits[-num_blocks:], gt)).mean()})")
    print_dbg(f" loss_weights :  {loss_weights}", verbose = True)
    print_dbg(f" cross_entropy:  {environ.cross_entropy2(logits[-num_blocks:], gt)}  ", verbose = True)
    print_dbg(f" loss[sparsity][{task_key}]: {self.losses['sparsity'][task_key] } ", verbose = True)

else:
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between \n Logits   : \n{logits[-num_blocks:]} \n and gt: \n{gt} \n", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-num_blocks:], gt)}")
    
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt[-1:])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt0[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt0[-1:])} \n")
    
    print('\n cond 3')    
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt[0:1])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt0[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt0[0:1])} \n")
        
        

cond 2
Compute CrossEntropyLoss between 
 Logits   : 
tensor([[0.3306, 0.4518],
        [0.3532, 0.5529],
        [0.3888, 0.6125],
        [0.4204, 0.7685],
        [0.4520, 0.7994],
        [0.4840, 0.8021]]) 
 and gt: 
tensor([1, 1, 1, 1, 1, 1]) 

task1_logits sparsity error:  0.5725929141044617

 cond 2
Compute CrossEntropyLoss between Logits      : tensor([[0.4840, 0.8021]])  and gt: 1 
task1_logits sparsity error:  0.5467103123664856 

Compute CrossEntropyLoss between Logits      : tensor([[0.4840, 0.8021]])  and gt: 0 
task1_logits sparsity error:  0.864768385887146 


 cond 3
Compute CrossEntropyLoss between Logits   : tensor([[0.3306, 0.4518]])  and gt: tensor([1]) 
task1_logits sparsity error:  0.634384036064148 

Compute CrossEntropyLoss between Logits   : tensor([[0.3306, 0.4518]])  and gt: tensor([0]) 
task1_logits sparsity error:  0.7555801868438721 



In [24]:
# flag = 'update_w'
# environ.fix_alpha
# environ.free_w(opt['fix_BN'])

flag = 'update_alpha'
environ.fix_weights()
environ.free_alpha()

In [29]:
environ.networks['mtl-net'].num_layers

6

In [25]:
print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

train_total_epochs += 5

print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

current_iters         : 6580
curr_epochs           : 60
train_total_epochs    : 60


In [31]:
environ.schedulers['weights'].get_last_lr()

[1e-05, 1e-05]

In [86]:
current_state = {}
for k, v in environ.optimizers.items():
    print(f'state dict for {k} = {v}')
    current_state[k] = v.state_dict()
pp.pprint(current_state)

state dict for weights = SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0001
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001

Parameter Group 1
    dampening: 0
    lr: 0.0001
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
)
state dict for alphas = Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0.0005
)
{   'alphas': {   'param_groups': [   {   'amsgrad': False,
                                          'betas': (0.9, 0.999),
                                          'eps': 1e-08,
                                          'lr': 0.0001,
                                          'params': [0, 1, 2],
                                          'weight_decay': 0.0005}],
                  'state': {   0: {   'exp_avg': tensor([[ 0.0607, -0.0007],
        [-0.0428, -0.0069],
        [-0.1218,  0.0138],
        [ 0.0086,  0.0238]], device='cuda:0'),
                                      'exp_

In [88]:
current_state = {}
for k, v in environ.schedulers.items():
    print(f'state dict for {k} = {v}')
    print(v.state_dict())

state dict for weights = <torch.optim.lr_scheduler.StepLR object at 0x7f90c01c0ca0>
{'step_size': 4000, 'gamma': 0.5, 'base_lrs': [0.0001, 0.0001], 'last_epoch': 9100, '_step_count': 9101, 'verbose': False, '_get_lr_called_within_step': False, '_last_lr': [2.5e-05, 2.5e-05]}


### Policy / Logit stuff

In [93]:
from scipy.special          import softmax

In [33]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=8,linewidth=132)

#### `get_task_logits(n)` Get logits for task group n

In [128]:
task_logits = environ.get_task_logits(1)
print(task_logits)

Parameter containing:
tensor([[-0.00035114, -0.06397165],
        [ 0.00056738, -0.03663344],
        [ 0.00056098, -0.02617791],
        [-0.00044851, -0.07137010],
        [ 0.00013184, -0.05879313],
        [ 0.00079021, -0.05743587]], device='cuda:0')


#### `get_arch_parameters()`: Get last used logits from network

In [34]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

[Parameter containing:
tensor([[-0.00035120, -0.06617914],
        [ 0.00056736, -0.04341661],
        [ 0.00056091, -0.01096974],
        [-0.00044879, -0.01083876],
        [ 0.00013163,  0.00874004],
        [ 0.00079006, -0.00861552]], device='cuda:0'), Parameter containing:
tensor([[-0.00035114, -0.06397165],
        [ 0.00056738, -0.03663344],
        [ 0.00056098, -0.02617791],
        [-0.00044851, -0.07137010],
        [ 0.00013184, -0.05879313],
        [ 0.00079021, -0.05743587]], device='cuda:0'), Parameter containing:
tensor([[-0.00035016, -0.06321616],
        [ 0.00056696, -0.03072025],
        [ 0.00056129, -0.01022454],
        [-0.00044983, -0.00021709],
        [ 0.00013071,  0.00484093],
        [ 0.00078938, -0.02230957]], device='cuda:0')]


In [25]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

[Parameter containing:
tensor([[ 1.873275e-03, -5.276022e-01],
        [ 2.345233e-03, -2.740704e-01],
        [ 2.614364e-03,  1.604760e-02],
        [ 2.143114e-04,  2.198091e-02],
        [ 4.191113e-04,  5.969038e-02],
        [ 2.007700e-03,  3.544179e-02]], device='cuda:0'), Parameter containing:
tensor([[ 1.873281e-03, -4.892288e-01],
        [ 2.345207e-03, -2.255457e-01],
        [ 2.614349e-03, -2.191145e-01],
        [ 2.143144e-04, -3.354620e-01],
        [ 4.190930e-04, -3.310193e-01],
        [ 2.007697e-03, -2.532191e-01]], device='cuda:0'), Parameter containing:
tensor([[ 1.873283e-03, -6.248206e-01],
        [ 2.345208e-03, -2.149665e-01],
        [ 2.614360e-03, -1.423603e-01],
        [ 2.143196e-04, -1.089546e-01],
        [ 4.191188e-04, -7.532501e-02],
        [ 2.007698e-03, -1.407905e-01]], device='cuda:0')]


#### `get_all_task_logits()`:  Get Policy Logits - returns same as `get_arch_parameters()`

In [26]:
logs = environ.get_all_task_logits()
for i in logs:
    print(i, '\n')
# probs = softmax(logs, axis= -1)
# for i in probs:
#     print(i, '\n')

[[ 1.8732749e-03 -5.2760220e-01]
 [ 2.3452332e-03 -2.7407044e-01]
 [ 2.6143640e-03  1.6047601e-02]
 [ 2.1431143e-04  2.1980910e-02]
 [ 4.1911125e-04  5.9690382e-02]
 [ 2.0077003e-03  3.5441790e-02]] 

[[ 1.8732806e-03 -4.8922884e-01]
 [ 2.3452067e-03 -2.2554573e-01]
 [ 2.6143489e-03 -2.1911447e-01]
 [ 2.1431442e-04 -3.3546203e-01]
 [ 4.1909298e-04 -3.3101928e-01]
 [ 2.0076970e-03 -2.5321913e-01]] 

[[ 1.8732828e-03 -6.2482059e-01]
 [ 2.3452076e-03 -2.1496648e-01]
 [ 2.6143598e-03 -1.4236034e-01]
 [ 2.1431963e-04 -1.0895463e-01]
 [ 4.1911882e-04 -7.5325012e-02]
 [ 2.0076977e-03 -1.4079048e-01]] 



#### `get_policy_prob()` : Gets the softmax of the logits

In [27]:
policy_softmaxs = environ.get_policy_prob()
for i in policy_softmaxs:
    print(i, '\n')

[[0.6293608  0.37063923]
 [0.5686673  0.43133274]
 [0.49664173 0.5033582 ]
 [0.4945586  0.5054415 ]
 [0.48518652 0.5148135 ]
 [0.49164233 0.50835776]] 

[[0.62036604 0.379634  ]
 [0.55672747 0.44327256]
 [0.5552062  0.44479376]
 [0.58313996 0.4168601 ]
 [0.58210933 0.41789067]
 [0.5634626  0.4365374 ]] 

[[0.6517394  0.34826055]
 [0.5541151  0.44588488]
 [0.5361803  0.46381968]
 [0.5272652  0.47273484]
 [0.518927   0.48107296]
 [0.535639   0.464361  ]] 



#### `get_sample_policy( hard_sampling = False)` : Calls test_sample_policy of network with random choices based on softmax of logits

In [382]:
policy_softmaxs = environ.get_policy_prob()
policies,logits = environ.get_sample_policy(hard_sampling = False)

for l, p, s in zip(logits, policies, policy_softmaxs) :
    for  l_row, p_row, s_row in zip(l, p, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

[-0.00035120 -0.06617914] 	 [0 1] 	 [0.51645106 0.48354897]
[ 0.00056736 -0.04341661] 	 [0 1] 	 [0.51099426 0.48900577]
[ 0.00056091 -0.01096974] 	 [1 0] 	 [0.50288266 0.49711737]
[-0.00044879 -0.01083876] 	 [0 1] 	 [0.50259751 0.49740252]
[0.00013163 0.00874004] 	 [0 1] 	 [0.49784794 0.50215214]
[ 0.00079006 -0.00861552] 	 [0 1] 	 [0.50235140 0.49764863]


[-0.00035114 -0.06397165] 	 [0 1] 	 [0.51589972 0.48410025]
[ 0.00056738 -0.03663344] 	 [1 0] 	 [0.50929916 0.49070087]
[ 0.00056098 -0.02617791] 	 [1 0] 	 [0.5066843 0.4933157]
[-0.00044851 -0.07137010] 	 [0 1] 	 [0.51772296 0.48227707]
[ 0.00013184 -0.05879313] 	 [0 1] 	 [0.514727 0.485273]
[ 0.00079021 -0.05743587] 	 [0 1] 	 [0.51455247 0.48544762]


[-0.00035016 -0.06321616] 	 [0 1] 	 [0.51571137 0.48428872]
[ 0.00056696 -0.03072025] 	 [1 0] 	 [0.50782120 0.49217883]
[ 0.00056129 -0.01022454] 	 [1 0] 	 [0.50269639 0.49730355]
[-0.00044983 -0.00021709] 	 [0 1] 	 [0.4999418 0.5000582]
[0.00013071 0.00484093] 	 [1 0] 	 [0.49882248 

#### `gumbel_softmax()`  

In [170]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', floatmode = 'maxprec_equal')
torch.set_printoptions(precision=8,linewidth=132)

In [319]:
print(environ.temp)
# tau = environ.temp
tau = 1
for i in range(3): 
    logits_tensor = torch.tensor(logits[0])
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=tau, hard=False).cpu().numpy() 

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=tau, hard=True).cpu().numpy()
    
    for l, gs, gh in zip(lgts, gumbel_soft, gumbel_hard):
        print(f"   {l}   \t {gs}            \t {gh}")
#     print(lgts)
#     print(gumbel_soft)
#     print(gumbel_hard)
    print()

0.0001180506617226113
   [-0.00035016 -0.06321616]   	 [0.17155227 0.82844770]            	 [1. 0.]
   [ 0.00056696 -0.03072025]   	 [0.04782803 0.95217192]            	 [1. 0.]
   [ 0.00056129 -0.01022454]   	 [0.52678031 0.47321963]            	 [0. 1.]
   [-0.00044983 -0.00021709]   	 [0.74226642 0.25773358]            	 [0. 1.]
   [0.00013071 0.00484093]   	 [0.81233245 0.18766758]            	 [1. 0.]
   [ 0.00078938 -0.02230957]   	 [0.76270294 0.23729712]            	 [1. 0.]

   [-0.00035016 -0.06321616]   	 [0.01975500 0.98024493]            	 [0. 1.]
   [ 0.00056696 -0.03072025]   	 [0.33801472 0.66198522]            	 [1. 0.]
   [ 0.00056129 -0.01022454]   	 [0.2644645 0.7355355]            	 [0. 1.]
   [-0.00044983 -0.00021709]   	 [0.08984101 0.91015899]            	 [1. 0.]
   [0.00013071 0.00484093]   	 [0.17066659 0.82933342]            	 [1. 0.]
   [ 0.00078938 -0.02230957]   	 [0.74648136 0.25351864]            	 [0. 1.]

   [-0.00035016 -0.06321616]   	 [0.5077298 0.